## Analysis of EV Charging Station Availability for a Roadtrip Through the USA

##### Electric vehicles are starting to penetrate the consumer auto market. One significant barrier to their adoption by consumers is the availability of charging stations for non-local excursions. Electric cars have ranges of 300-500 miles, so having confidence in being able to find a charging port while planning any significant road trip becomes rather difficult. Obtaining insight into the availability of charging ports at major cities - and their proximity to hotels - would provide a major boost of confidence to the electric vehicle consumer. This data could also be used to help determine where a new charging station may be best suited to maximize availability for consumers.

##### FourSquare includes in its a database a list of EV charging locations. To help build confidence in a road trip plan, this data can be used and combined with FourSquare's city hotel data to help plan an itinerary that includes "distance to charging port" as a main criterion, along with typical decisional factors like hotel rating and location. For each major city on the route, the location of hotels can be visualized by using clustering to find highly rated hotels which have a charging station within walking distance. Performing this exercise at each major city stopping point along the way will yield a road trip itinerary that ensures adequate and convenient charging capability for an electric vehicle.



#### Install and import libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 102kB 9.0MB/s ta 0:00:011
Folium installed
Libraries imported.


#### Specify foursquare API credentials

In [2]:
CLIENT_ID = 'UJV5PXV1XQOWNEWWMKLKAV52BC0NUD3NLCSMH013P112YYPR' # your Foursquare ID
CLIENT_SECRET = 'EIU5V1AYEG032Y0V03DEHRHVNHG2DVG0F1XPP022SPLXVM0T' # your Foursquare Secret
VERSION = '20200720' # Foursquare API version
LIMIT = 100

#### Specify city waypoints for cross country trip and obtain coordinates using geolocator

In [3]:
address = ['Los Angeles, CA', 'Las Vegas, NV', 'Provo, UT', 'Denver, CO', 'Topeka, KS', 'St. Louis, MO', 'Cincinnati, OH']
latitude_list = []
longitude_list = []

for add in address:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(add)
    latitude_list.append(location.latitude)
    longitude_list.append(location.longitude)
    #print(latitude, longitude)

#### Specify radius limits and category criteria for API query

In [4]:
radius_hotel = 12000.0
radius_charging_limit = 1500.0

hotel_category_id = '4bf58dd8d48988d1fa931735'
charging_category_id = '5032872391d4c4b30a586d64'



#### Use the first city (LA) as the example for this exercise. Foursquare does not allow enough free API calls to evalute all the cities on the route

#### Generate the API url and execute the query to get a list of hotels within 12000 meters of the city center. Save the results

In [5]:
latitude = latitude_list[0]
longitude = longitude_list[0]

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, hotel_category_id, radius_hotel, LIMIT)

In [6]:
results_hotel = requests.get(url).json()


#### Obtain the relevant columns and save to a dataframe, and clean up the column names

In [7]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']

# tranform venues into a dataframe
dataframe_hotel = json_normalize(venues_hotel)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_hotel.columns if col.startswith('location.')] + ['id']
dataframe_hotel_filtered = dataframe_hotel.loc[:, filtered_columns]

In [8]:


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_hotel_filtered['categories'] = dataframe_hotel_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_hotel_filtered.columns = [column.split('.')[-1] for column in dataframe_hotel_filtered.columns]
dataframe_hotel_filtered['Rating']=np.nan
dataframe_hotel_filtered['Stations_Count_1500']=np.nan
dataframe_hotel_filtered['Stations_Count_1000']=np.nan
dataframe_hotel_filtered['Stations_Count_500']=np.nan
dataframe_hotel_filtered['Station Min Distance']=np.nan

initialize = ['a','b','c']

dataframe_hotel_filtered['Stations_Lat_1500']=str(initialize)
dataframe_hotel_filtered['Stations_Lat_1000']=str(initialize)
dataframe_hotel_filtered['Stations_Lat_500']=str(initialize)
dataframe_hotel_filtered['Stations_Lng_1500']=str(initialize)
dataframe_hotel_filtered['Stations_Lng_1000']=str(initialize)
dataframe_hotel_filtered['Stations_Lng_500']=str(initialize)


#### For each hotel in the list, perform an API query to get a list of charging stations within 1500 meters of the hotel

#### Count the number of stations within 500, 1000, and 1500 meters of the hotel, and save the list of coordinates of these stations to a string column in the dataframe

In [9]:
radius_charging_limit = 1500.0

Hotel_Lat_List = dataframe_hotel_filtered['lat'].tolist()
Hotel_Lng_List = dataframe_hotel_filtered['lng'].tolist()
Hotel_id_List = dataframe_hotel_filtered['id'].tolist()

for Latitude, Longitude, venue_id in zip(Hotel_Lat_List,Hotel_Lng_List,Hotel_id_List):
    
    url_hotel_venue = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result_hotel_venue = requests.get(url_hotel_venue).json()
    print(result_hotel_venue)
    try:
        rating = result_hotel_venue['response']['venue']['rating']
    except:
        rating = np.nan
    
    print(rating)
    dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Rating'] = rating

    url_charging = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Latitude, Longitude, VERSION, charging_category_id, radius_charging_limit, LIMIT)
    results_charging_station = requests.get(url_charging).json()

    # assign relevant part of JSON to venues
    try:
        venues_charging_station = results_charging_station['response']['venues']
        # tranform venues into a dataframe
        dataframe_charging_station = json_normalize(venues_charging_station)
    except:
        print('exception')
        dataframe_charging_station = pd.DataFrame(columns=[])
    
    print(dataframe_charging_station.size)
    if dataframe_charging_station.size > 0:
        # keep only columns that include venue name, and anything that is associated with location
        filtered_columns = ['name', 'categories'] + [col for col in dataframe_charging_station.columns if col.startswith('location.')] + ['id']
        dataframe_charging_station_filtered = dataframe_charging_station.loc[:, filtered_columns]
        
        # filter the category for each row
        dataframe_charging_station_filtered['categories'] = dataframe_charging_station_filtered.apply(get_category_type, axis=1)

        # clean column names by keeping only last term
        dataframe_charging_station_filtered.columns = [column.split('.')[-1] for column in dataframe_charging_station_filtered.columns]
        
        dataframe_charging_station_filtered_1500 = dataframe_charging_station_filtered[dataframe_charging_station_filtered['distance'] <= 1500]
        dataframe_charging_station_filtered_1000 = dataframe_charging_station_filtered[dataframe_charging_station_filtered['distance'] <= 1000]
        dataframe_charging_station_filtered_500 = dataframe_charging_station_filtered[dataframe_charging_station_filtered['distance'] <= 500]
        
        count_1500 = dataframe_charging_station_filtered_1500.id.count()
        count_1000 = dataframe_charging_station_filtered_1000.id.count()
        count_500 = dataframe_charging_station_filtered_500.id.count()

        lat_1500 = dataframe_charging_station_filtered_1500['lat'].tolist()
        lng_1500 = dataframe_charging_station_filtered_1500['lng'].tolist()

        lat_1000 = dataframe_charging_station_filtered_1000['lat'].tolist()
        lng_1000 = dataframe_charging_station_filtered_1000['lng'].tolist()

        lat_500 = dataframe_charging_station_filtered_500['lat'].tolist()
        lng_500 = dataframe_charging_station_filtered_500['lng'].tolist()
        
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Count_1500'] = count_1500
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Count_1000'] = count_1000
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Count_500'] = count_500

        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lat_1500'] = str(lat_1500)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lat_1000'] = str(lat_1000)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lat_500'] = str(lat_500)

        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lng_1500'] = str(lng_1500)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lng_1000'] = str(lng_1000)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lng_500'] = str(lng_500)    
        
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Station Min Distance'] = min(dataframe_charging_station_filtered['distance'])
        

{'meta': {'code': 200, 'requestId': '5f2710842378a6144ba01a3c'}, 'response': {'venue': {'id': '4a6b9a88f964a52052cf1fe3', 'name': 'Millennium Biltmore Hotel Los Angeles', 'contact': {'phone': '2136241011', 'formattedPhone': '(213) 624-1011', 'twitter': 'millennium_usa', 'facebook': '77473804100', 'facebookUsername': 'MillenniumHotelsNorthAmerica', 'facebookName': 'Millennium Hotels and Resorts North America'}, 'location': {'address': '506 S Grand Ave', 'crossStreet': 'at 5th St', 'lat': 34.049813, 'lng': -118.253938, 'labeledLatLngs': [{'label': 'display', 'lat': 34.049813, 'lng': -118.253938}], 'postalCode': '90071', 'cc': 'US', 'city': 'Los Angeles', 'state': 'CA', 'country': 'United States', 'formattedAddress': ['506 S Grand Ave (at 5th St)', 'Los Angeles, CA 90071', 'United States']}, 'canonicalUrl': 'https://foursquare.com/v/millennium-biltmore-hotel-los-angeles/4a6b9a88f964a52052cf1fe3', 'categories': [{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 's

#### Clean up the dataframe by removing uneeded columns and removing null rows

In [10]:
dataframe_hotel_filtered.dropna(subset=['Rating','Stations_Count_1500'],inplace=True)
dataframe_hotel_filtered.drop(columns=['country','cc','categories','crossStreet','formattedAddress','labeledLatLngs','neighborhood','postalCode','state'],inplace=True)
#dataframe_hotel_filtered = dataframe_hotel_filtered[dataframe_hotel_filtered.Stations_Count_1500 > 0]

In [31]:
dataframe_hotel_filtered.head(20)
dataframe_hotel_filtered = dataframe_hotel_filtered[dataframe_hotel_filtered['Station Min Distance']<1501]
dataframe_hotel_filtered.head(20)
dataframe_hotel_filtered.drop(columns=['id'],inplace=True)
dataframe_hotel_filtered.head(20)

,name,address,city,distance,lat,lng,Rating,Stations_Count_1500,Stations_Count_1000,Stations_Count_500,Station Min Distance,Stations_Lat_1500,Stations_Lat_1000,Stations_Lat_500,Stations_Lng_1500,Stations_Lng_1000,Stations_Lng_500
0,Millennium Biltmore Hotel Los Angeles,506 S Grand Ave,Los Angeles,1117,34.049813,-118.253938,7.3,1.0,0.0,0.0,1019.0,[34.047691345214844],[],[],[-118.26469421386719],[],[]
3,Home2 Suites by Hilton,988 Via San Clemente,Montebello,10737,34.029770,-118.130001,8.2,2.0,2.0,1.0,246.0,"[34.032108306884766, 34.02803039550781]","[34.032108306884766, 34.02803039550781]",[34.02803039550781],"[-118.1213607788086, -118.13166046142578]","[-118.1213607788086, -118.13166046142578]",[-118.13166046142578]
18,Freehand Los Angeles,416 W 8th St,Los Angeles,1603,34.044964,-118.256593,9.0,1.0,1.0,0.0,806.0,[34.047691345214844],[34.047691345214844],[],[-118.26469421386719],[-118.26469421386719],[]
20,Soho Warehouse,1000 S Santa Fe Ave,Los Angeles,2744,34.031513,-118.229770,9.0,1.0,1.0,0.0,520.0,[34.03594970703125],[34.03594970703125],[],[-118.23155975341797],[-118.23155975341797],[]
24,Ace Hotel Downtown Los Angeles,929 S Broadway,Los Angeles,1843,34.041763,-118.256630,8.8,2.0,1.0,0.0,994.0,"[34.047691345214844, 34.03177]",[34.047691345214844],[],"[-118.26469421386719, -118.247257]",[-118.26469421386719],[]
32,Hotel Indigo,899 Francisco St,Los Angeles,2113,34.047450,-118.264410,8.5,1.0,1.0,1.0,37.0,[34.047691345214844],[34.047691345214844],[34.047691345214844],[-118.26469421386719],[-118.26469421386719],[-118.26469421386719]
33,"The Westin Bonaventure Hotel & Suites, Los Ang...",404 S Figueroa St,Los Angeles,1187,34.052777,-118.255596,7.7,1.0,0.0,0.0,1012.0,[34.047691345214844],[],[],[-118.26469421386719],[],[]
36,Embassy Suites by Hilton,800 N Central Ave,Glendale,11657,34.157699,-118.257496,7.5,2.0,0.0,0.0,1322.0,"[34.146005, 34.14479820099191]",[],[],"[-118.260037, -118.2570037028243]",[],[]
37,The Mayfair Hotel,1256 W 7th Street,Los Angeles,2319,34.052021,-118.267832,5.2,1.0,1.0,0.0,562.0,[34.047691345214844],[34.047691345214844],[],[-118.26469421386719],[-118.26469421386719],[]
38,Sheraton Grand Los Angeles (Sheraton Los Angel...,711 S Hope St,Los Angeles,1579,34.047863,-118.258376,6.9,1.0,1.0,0.0,583.0,[34.047691345214844],[34.047691345214844],[],[-118.26469421386719],[-118.26469421386719],[]


In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lon, rating, closest_station, name in zip(dataframe_hotel_filtered['lat'], dataframe_hotel_filtered['lng'], dataframe_hotel_filtered['Rating'],dataframe_hotel_filtered['Station Min Distance'],dataframe_hotel_filtered['name']):
    label = folium.Popup(str(name) + '\n Rating =  ' + str(rating) + '\n Closest Station = ' + str(closest_station), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

for lat, lon, station_500 in zip(dataframe_hotel_filtered['lat'], dataframe_hotel_filtered['lng'], dataframe_hotel_filtered['Stations_Count_500']):
    if station_500 > 0:
        #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=7,
            #popup=label,
            color='green',
            fill=False,
            #fill_color='',
            #fill_opacity=0.0).add_to(map_clusters)
            ).add_to(map_clusters)

for lat, lon, station_1000 in zip(dataframe_hotel_filtered['lat'], dataframe_hotel_filtered['lng'], dataframe_hotel_filtered['Stations_Count_1000']):
    if station_1000 > 0:
        #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=10,
            #popup=label,
            color='yellow',
            fill=False,
            #fill_color='',
            #fill_opacity=0.0).add_to(map_clusters)
            ).add_to(map_clusters)

for lat, lon, station_1500 in zip(dataframe_hotel_filtered['lat'], dataframe_hotel_filtered['lng'], dataframe_hotel_filtered['Stations_Count_1500']):
    if station_1500 > 0:
        #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=13,
            #popup=label,
            color='red',
            fill=False,
            #fill_color='',
            #fill_opacity=0.0).add_to(map_clusters)
            ).add_to(map_clusters)
        
        
map_clusters

